In [ ]:
!pip install dotmap
!pip install keras-flops

In [ ]:
from keras.datasets import cifar10
from keras.models import Sequential
from keras.layers import Input, Dense, Conv2D, MaxPooling2D, Dropout, Flatten
import os
import tensorflow as tf
from keras.layers import Activation, Input, Dense, Conv2D, MaxPooling2D, Dropout, Flatten, BatchNormalization
from keras.layers import Conv2D, MaxPooling2D, GlobalAveragePooling2D
from keras.callbacks import ModelCheckpoint, TensorBoard
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import time
import json
from dotmap import DotMap
from skimage.transform import resize
from keras_flops import get_flops
from sklearn.metrics import confusion_matrix
import itertools
from tensorflow.keras.applications import ResNet50, Xception, InceptionV3

In [ ]:
os.chdir("/content/drive/MyDrive/Design_Credit_Last_Sem")

In [ ]:
def image_resize(images):
  temp=[]
  i=0
  for image in images:
    print("i", i)
    image=resize(image, (150,150))
    temp.append(image)
    i=i+1
  temp=np.array(temp)
  return(temp)

In [ ]:
class ConvCifarDataLoader():
    def __init__(self):
        (self.X_train, self.y_train), (self.X_test, self.y_test) = cifar10.load_data()
        self.X_train=image_resize(self.X_train[:1000])
        self.X_test=image_resize(self.X_test[:100])
        self.y_train=self.y_train.reshape(-1,)
        self.y_train=self.y_train[:1000]
        self.y_test=self.y_test.reshape(-1,)
        self.y_test=self.y_test[:100]

    def get_train_data(self):
        return self.X_train, self.y_train

    def get_test_data(self):
        return self.X_test, self.y_test

In [ ]:
def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues, name="temp"):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]

    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, cm[i, j],
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

    plt.savefig(name+"_confusion"+".png")

In [ ]:
class SimpleCifarModelTrainer():
    def __init__(self, model, data, config):
        self.model=model
        self.data=data
        self.config=config
        self.callbacks = []
        self.loss = []
        self.acc = []
        self.top5=[]
        self.val_loss = []
        self.val_acc = []
        self.val_top5=[] 
        self.init_callbacks()

    def init_callbacks(self):
        self.callbacks.append(
            ModelCheckpoint(
                filepath=os.path.join(self.config.callbacks.checkpoint_dir, '%s-{epoch:02d}-{val_loss:.2f}.hdf5' % self.config.exp.name),
                monitor=self.config.callbacks.checkpoint_monitor,
                mode=self.config.callbacks.checkpoint_mode,
                save_best_only=self.config.callbacks.checkpoint_save_best_only,
                save_weights_only=self.config.callbacks.checkpoint_save_weights_only,
                verbose=self.config.callbacks.checkpoint_verbose,
            )
        )

        self.callbacks.append(
            TensorBoard(
                log_dir=self.config.callbacks.tensorboard_log_dir,
                write_graph=self.config.callbacks.tensorboard_write_graph,
            )
        )


    def train(self):
        history = self.model.fit(
            self.data[0], self.data[1],
            epochs=self.config.trainer.num_epochs,
            verbose=self.config.trainer.verbose_training,
            batch_size=self.config.trainer.batch_size,
            validation_split=self.config.trainer.validation_split,
            callbacks=self.callbacks,
        )
        self.loss.extend(history.history['loss'])
        self.acc.extend(history.history['accuracy'])
        self.val_top5.extend(history.history['top_k_categorical_accuracy'])
        self.val_loss.extend(history.history['val_loss'])
        self.val_acc.extend(history.history['val_accuracy'])
        self.val_top5.extend(history.history['val_top_k_categorical_accuracy'])

In [ ]:
class simpleCifarTester():
    def __init__(self, model, data, config, training_time, flops):
        self.model=model
        self.data=data
        self.config=config
        self.result=[]
    def test(self):
        print("Evaluating on test data")
        results = self.model.evaluate(self.data[0], self.data[1])
        print("test results: ", results)
        print("Saving the test reults metrics: ")
        results_model=np.array(results)
        np.save(self.config.callbacks.testPredictions_dir+"results_model.npy", results_model)
        results_performance=np.array([training_time, flops])
        np.save(self.config.callbacks.testPredictions_dir+"results_performance.npy", results_performance)
        y_prediction = np.argmax(self.model.predict(self.data[0]), axis=1)
        y_true=self.data[1]
        result = confusion_matrix(y_true, y_prediction)
        self.result=result

In [ ]:
config={
  "exp": {
    "name": "conv_Cifar_from_config"
  },
  "model":{
    "learning_rate": 0.002,
    "optimizer": "adam"
  },
  "trainer":{
    "num_epochs": 1,
    "batch_size": 32,
    "validation_split":0.2,
    "verbose_training": True
  },
  "callbacks":{
    "checkpoint_monitor": "val_loss",
    "checkpoint_mode": "min",
    "checkpoint_save_best_only": True,
   "checkpoint_save_weights_only": True,
    "checkpoint_verbose": True,
    "tensorboard_write_graph": True
  }
}
config = DotMap(config)

In [ ]:
def create_dirs(dirs):
    """
    dirs - a list of directories to create if these directories are not found
    :param dirs:
    :return exit_code: 0:success -1:failed
    """
    try:
        for dir_ in dirs:
            if not os.path.exists(dir_):
                os.makedirs(dir_)
        return 0
    except Exception as err:
        print("Creating directories error: {0}".format(err))
        exit(-1)

Simple cnn

In [ ]:
class ConvCifarModel():
    def __init__(self):
        self.build_model()

    def build_model(self):
        self.model = Sequential()
        self.model.add(Conv2D(32, kernel_size=(4, 4),
                        #  activation='relu', input_shape=(28, 28, 1)))
                         activation='relu', input_shape=(150, 150, 3)))
        self.model.add(MaxPooling2D(pool_size=(2, 2)))
        self.model.add(Conv2D(64, (4, 4), activation='relu'))
        self.model.add(MaxPooling2D(pool_size=(2, 2)))
        self.model.add(Flatten())
        # self.model.add(Dropout(0.25))
        # self.model.add(Flatten())
        self.model.add(Dense(128, activation='relu'))
        self.model.add(Dropout(0.25))
        self.model.add(Dense(64, activation='relu'))
        self.model.add(Dropout(0.25))
        self.model.add(Dense(32, activation='relu'))
        self.model.add(Dropout(0.25))
        self.model.add(Dense(10, activation='softmax'))
        
model=ConvCifarModel()
model=model.model

In [ ]:
class ConvCifarModel():
    def __init__(self, config):
        self.config=config
        self.build_model()

    def build_model(self):
        self.model = Sequential()
        self.model.add(Conv2D(32, kernel_size=(4, 4),
                        #  activation='relu', input_shape=(28, 28, 1)))
                         activation='relu', input_shape=(150, 150, 3)))
        self.model.add(MaxPooling2D(pool_size=(2, 2)))
        self.model.add(Conv2D(64, (4, 4), activation='relu'))
        self.model.add(MaxPooling2D(pool_size=(2, 2)))
        self.model.add(Flatten())
        # self.model.add(Dropout(0.25))
        # self.model.add(Flatten())
        self.model.add(Dense(128, activation='relu'))
        self.model.add(Dropout(0.25))
        self.model.add(Dense(64, activation='relu'))
        self.model.add(Dropout(0.25))
        self.model.add(Dense(32, activation='relu'))
        self.model.add(Dropout(0.25))
        self.model.add(Dense(10, activation='softmax'))

        self.model.compile(
              loss='sparse_categorical_crossentropy',
              optimizer=self.config.model.optimizer,
              metrics=['accuracy', 'top_k_categorical_accuracy'])
              #metrics=['accuracy'])

config.exp.name="conv_cifar_from_config"
config.callbacks.tensorboard_log_dir = os.path.join("experiments", time.strftime("%Y-%m-%d/",time.localtime()), config.exp.name, "logs/")
config.callbacks.checkpoint_dir = os.path.join("experiments", time.strftime("%Y-%m-%d/",time.localtime()), config.exp.name, "checkpoints/")
config.callbacks.testPredictions_dir=os.path.join("experiments", time.strftime("%Y-%m-%d/",time.localtime()), config.exp.name, "testPredictions/")
create_dirs([config.callbacks.testPredictions_dir, config.callbacks.tensorboard_log_dir, config.callbacks.checkpoint_dir])

print('Create the data generator.')
data_loader = ConvCifarDataLoader()
print('Create the model.')
model = ConvCifarModel(config)
print('Create the trainer')
trainer = SimpleCifarModelTrainer(model.model, data_loader.get_train_data(), config)
print('Start training the model.')
t0 = time.time()
trainer.train()
training_time=time.time()-t0


print("*******************************************************************************")
print(model.model.summary())


print("*******************************************************************************")
print("Training_time: ", training_time)


print("*******************************************************************************")
flops = get_flops(model.model, batch_size=1)
print(f"FLOPS: {flops / 10 ** 9:.03} G")


print("*******************************************************************************")
print("Create the tester")
tester = simpleCifarTester(model.model, data_loader.get_test_data(), config, training_time, flops)     
print("Start testing the model.")
tester.test()


In [ ]:
from keras.utils.vis_utils import plot_model
plot_model(model.model, to_file=config.exp.name+'.jpg', show_shapes=True, show_layer_names=True)

In [ ]:
print("*******************************************************************************")
# Plot the loss and accuracy curves for training and validation 
fig, ax = plt.subplots(2,1)
ax[0].plot(trainer.loss, color='b', label="Training loss")
ax[0].plot(trainer.val_loss, color='r', label="validation loss",axes =ax[0])
legend = ax[0].legend(loc='best', shadow=True)
ax[1].plot(trainer.acc, color='b', label="Training accuracy")
ax[1].plot(trainer.val_acc, color='r',label="Validation accuracy")
legend = ax[1].legend(loc='best', shadow=True)
plt.savefig(config.exp.name+".png")

In [ ]:
print("*******************************************************************************")
print(tester.result)

print("*******************************************************************************")
plot_confusion_matrix(tester.result, classes=range(10), name=config.exp.name)

Resnet

In [ ]:
class ResnetCifarModel():
    def __init__(self, config):
        self.config=config
        self.build_model()

    def build_model(self):
        self.model = Sequential()
        self.model.add(ResNet50(include_top=False, weights=None, input_tensor=None, input_shape=(150, 150, 3), pooling=None))
        self.model.add(GlobalAveragePooling2D())
        self.model.add(Dense(128))
        self.model.add(Activation("relu"))
        self.model.add(BatchNormalization(axis=-1, momentum=0.99, epsilon=0.001, center=True, scale=True, beta_initializer='zeros', gamma_initializer='ones', moving_mean_initializer='zeros', moving_variance_initializer='ones', beta_regularizer=None, gamma_regularizer=None, beta_constraint=None, gamma_constraint=None))
        self.model.add(Dense(64))
        self.model.add(Activation("relu"))
        self.model.add(Dense(32))
        self.model.add(Activation("relu"))
        self.model.add(Dense(10))
        self.model.add(Activation("softmax"))

        self.model.compile(
              loss='sparse_categorical_crossentropy',
              optimizer=self.config.model.optimizer,
              metrics=['accuracy', 'top_k_categorical_accuracy'])
              #metrics=['accuracy'])

config.exp.name="resnet_cifar_from_config"
config.callbacks.tensorboard_log_dir = os.path.join("experiments", time.strftime("%Y-%m-%d/",time.localtime()), config.exp.name, "logs/")
config.callbacks.checkpoint_dir = os.path.join("experiments", time.strftime("%Y-%m-%d/",time.localtime()), config.exp.name, "checkpoints/")
config.callbacks.testPredictions_dir=os.path.join("experiments", time.strftime("%Y-%m-%d/",time.localtime()), config.exp.name, "testPredictions/")
create_dirs([config.callbacks.testPredictions_dir, config.callbacks.tensorboard_log_dir, config.callbacks.checkpoint_dir])

print('Create the data generator.')
data_loader = ConvCifarDataLoader()
print('Create the model.')
model = ResnetCifarModel(config)
print('Create the trainer')
trainer = SimpleCifarModelTrainer(model.model, data_loader.get_train_data(), config)
print('Start training the model.')
t0 = time.time()
trainer.train()
training_time=time.time()-t0

print("*******************************************************************************")
print(model.model.summary())


print("*******************************************************************************")
print("Training_time: ", training_time)


print("*******************************************************************************")
flops = get_flops(model.model, batch_size=1)
print(f"FLOPS: {flops / 10 ** 9:.03} G")


print("*******************************************************************************")
print("Create the tester")
tester = simpleCifarTester(model.model, data_loader.get_test_data(), config, training_time, flops)     
print("Start testing the model.")
tester.test()

In [ ]:
from keras.utils.vis_utils import plot_model
plot_model(model.model, to_file=config.exp.name+'.png', show_shapes=True, show_layer_names=True)

In [ ]:
print("*******************************************************************************")
# Plot the loss and accuracy curves for training and validation 
fig, ax = plt.subplots(2,1)
ax[0].plot(trainer.loss, color='b', label="Training loss")
ax[0].plot(trainer.val_loss, color='r', label="validation loss",axes =ax[0])
legend = ax[0].legend(loc='best', shadow=True)
ax[1].plot(trainer.acc, color='b', label="Training accuracy")
ax[1].plot(trainer.val_acc, color='r',label="Validation accuracy")
legend = ax[1].legend(loc='best', shadow=True)
plt.savefig(config.exp.name+".png")

In [ ]:
print("*******************************************************************************")
print(tester.result)

print("*******************************************************************************")
plot_confusion_matrix(tester.result, classes=range(10), name=config.exp.name)

Xception

In [ ]:
class XceptionCifarModel():
    def __init__(self, config):
        self.config=config
        self.build_model()

    def build_model(self):
        self.model = Sequential()
        self.model.add(Xception(include_top=False, weights=None, input_tensor=None, input_shape=(150, 150, 3), pooling=None))
        self.model.add(GlobalAveragePooling2D())
        self.model.add(Dense(128))
        self.model.add(Activation("relu"))
        self.model.add(BatchNormalization(axis=-1, momentum=0.99, epsilon=0.001, center=True, scale=True, beta_initializer='zeros', gamma_initializer='ones', moving_mean_initializer='zeros', moving_variance_initializer='ones', beta_regularizer=None, gamma_regularizer=None, beta_constraint=None, gamma_constraint=None))
        self.model.add(Dense(64))
        self.model.add(Activation("relu"))
        self.model.add(Dense(32))
        self.model.add(Activation("relu"))
        self.model.add(Dense(10))
        self.model.add(Activation("softmax"))

        self.model.compile(
              loss='sparse_categorical_crossentropy',
              optimizer=self.config.model.optimizer,
              metrics=['accuracy', 'top_k_categorical_accuracy'])
              #metrics=['accuracy'])

config.exp.name="xception_cifar_from_config"
config.callbacks.tensorboard_log_dir = os.path.join("experiments", time.strftime("%Y-%m-%d/",time.localtime()), config.exp.name, "logs/")
config.callbacks.checkpoint_dir = os.path.join("experiments", time.strftime("%Y-%m-%d/",time.localtime()), config.exp.name, "checkpoints/")
config.callbacks.testPredictions_dir=os.path.join("experiments", time.strftime("%Y-%m-%d/",time.localtime()), config.exp.name, "testPredictions/")
create_dirs([config.callbacks.testPredictions_dir, config.callbacks.tensorboard_log_dir, config.callbacks.checkpoint_dir])

print('Create the data generator.')
data_loader = ConvCifarDataLoader()
print('Create the model.')
model = XceptionCifarModel(config)
print('Create the trainer')
trainer = SimpleCifarModelTrainer(model.model, data_loader.get_train_data(), config)
print('Start training the model.')
t0 = time.time()
trainer.train()
training_time=time.time()-t0

print("*******************************************************************************")
print(model.model.summary())


print("*******************************************************************************")
print("Training_time: ", training_time)


print("*******************************************************************************")
flops = get_flops(model.model, batch_size=1)
print(f"FLOPS: {flops / 10 ** 9:.03} G")


print("*******************************************************************************")
print("Create the tester")
tester = simpleCifarTester(model.model, data_loader.get_test_data(), config, training_time, flops)     
print("Start testing the model.")
tester.test()

In [ ]:
from keras.utils.vis_utils import plot_model
plot_model(model.model, to_file=config.exp.name+'.png', show_shapes=True, show_layer_names=True)

In [ ]:
print("*******************************************************************************")
# Plot the loss and accuracy curves for training and validation 
fig, ax = plt.subplots(2,1)
ax[0].plot(trainer.loss, color='b', label="Training loss")
ax[0].plot(trainer.val_loss, color='r', label="validation loss",axes =ax[0])
legend = ax[0].legend(loc='best', shadow=True)
ax[1].plot(trainer.acc, color='b', label="Training accuracy")
ax[1].plot(trainer.val_acc, color='r',label="Validation accuracy")
legend = ax[1].legend(loc='best', shadow=True)
plt.savefig(config.exp.name+".png")

In [ ]:
print("*******************************************************************************")
print(tester.result)

print("*******************************************************************************")
plot_confusion_matrix(tester.result, classes=range(10), name=config.exp.name)

MobileNet

In [ ]:
from tensorflow.keras.applications import MobileNet
class MobileNetCifarModel():
    def __init__(self, config):
        self.config=config
        self.build_model()

    def build_model(self):
        self.model = Sequential()
        self.model.add(MobileNet(include_top=False, weights=None, input_tensor=None, input_shape=(150, 150, 3), pooling=None))
        self.model.add(GlobalAveragePooling2D())
        self.model.add(Dense(128))
        self.model.add(Activation("relu"))
        self.model.add(BatchNormalization(axis=-1, momentum=0.99, epsilon=0.001, center=True, scale=True, beta_initializer='zeros', gamma_initializer='ones', moving_mean_initializer='zeros', moving_variance_initializer='ones', beta_regularizer=None, gamma_regularizer=None, beta_constraint=None, gamma_constraint=None))
        self.model.add(Dense(64))
        self.model.add(Activation("relu"))
        self.model.add(Dense(32))
        self.model.add(Activation("relu"))
        self.model.add(Dense(10))
        self.model.add(Activation("softmax"))

        self.model.compile(
              loss='sparse_categorical_crossentropy',
              optimizer=self.config.model.optimizer,
              metrics=['accuracy', 'top_k_categorical_accuracy'])
              #metrics=['accuracy'])

config.exp.name="mobileNet_cifar_from_config"
config.callbacks.tensorboard_log_dir = os.path.join("experiments", time.strftime("%Y-%m-%d/",time.localtime()), config.exp.name, "logs/")
config.callbacks.checkpoint_dir = os.path.join("experiments", time.strftime("%Y-%m-%d/",time.localtime()), config.exp.name, "checkpoints/")
config.callbacks.testPredictions_dir=os.path.join("experiments", time.strftime("%Y-%m-%d/",time.localtime()), config.exp.name, "testPredictions/")
create_dirs([config.callbacks.testPredictions_dir, config.callbacks.tensorboard_log_dir, config.callbacks.checkpoint_dir])

print('Create the data generator.')
data_loader = ConvCifarDataLoader()
print('Create the model.')
model = MobileNetCifarModel(config)
print('Create the trainer')
trainer = SimpleCifarModelTrainer(model.model, data_loader.get_train_data(), config)
print('Start training the model.')
t0 = time.time()
trainer.train()
training_time=time.time()-t0

print("*******************************************************************************")
print(model.model.summary())


print("*******************************************************************************")
print("Training_time: ", training_time)


print("*******************************************************************************")
flops = get_flops(model.model, batch_size=1)
print(f"FLOPS: {flops / 10 ** 9:.03} G")


print("*******************************************************************************")
print("Create the tester")
tester = simpleCifarTester(model.model, data_loader.get_test_data(), config, training_time, flops)     
print("Start testing the model.")
tester.test()

In [ ]:
from keras.utils.vis_utils import plot_model
plot_model(model.model, to_file=config.exp.name+'.png', show_shapes=True, show_layer_names=True)

In [ ]:
print("*******************************************************************************")
# Plot the loss and accuracy curves for training and validation 
fig, ax = plt.subplots(2,1)
ax[0].plot(trainer.loss, color='b', label="Training loss")
ax[0].plot(trainer.val_loss, color='r', label="validation loss",axes =ax[0])
legend = ax[0].legend(loc='best', shadow=True)
ax[1].plot(trainer.acc, color='b', label="Training accuracy")
ax[1].plot(trainer.val_acc, color='r',label="Validation accuracy")
legend = ax[1].legend(loc='best', shadow=True)
plt.savefig(config.exp.name+".png")

In [ ]:
print("*******************************************************************************")
print(tester.result)

print("*******************************************************************************")
plot_confusion_matrix(tester.result, classes=range(10), name=config.exp.name)

MobileNet v2

In [ ]:
from tensorflow.keras.applications import MobileNetV2
class MobileNetV2CifarModel():
    def __init__(self, config):
        self.config=config
        self.build_model()

    def build_model(self):
        self.model = Sequential()
        self.model.add(MobileNetV2(include_top=False, weights=None, input_tensor=None, input_shape=(150, 150, 3), pooling=None))
        self.model.add(GlobalAveragePooling2D())
        self.model.add(Dense(128))
        self.model.add(Activation("relu"))
        self.model.add(BatchNormalization(axis=-1, momentum=0.99, epsilon=0.001, center=True, scale=True, beta_initializer='zeros', gamma_initializer='ones', moving_mean_initializer='zeros', moving_variance_initializer='ones', beta_regularizer=None, gamma_regularizer=None, beta_constraint=None, gamma_constraint=None))
        self.model.add(Dense(64))
        self.model.add(Activation("relu"))
        self.model.add(Dense(32))
        self.model.add(Activation("relu"))
        self.model.add(Dense(10))
        self.model.add(Activation("softmax"))

        self.model.compile(
              loss='sparse_categorical_crossentropy',
              optimizer=self.config.model.optimizer,
              metrics=['accuracy', 'top_k_categorical_accuracy'])
              #metrics=['accuracy'])

config.exp.name="mobileNetV2_cifar_from_config"
config.callbacks.tensorboard_log_dir = os.path.join("experiments", time.strftime("%Y-%m-%d/",time.localtime()), config.exp.name, "logs/")
config.callbacks.checkpoint_dir = os.path.join("experiments", time.strftime("%Y-%m-%d/",time.localtime()), config.exp.name, "checkpoints/")
config.callbacks.testPredictions_dir=os.path.join("experiments", time.strftime("%Y-%m-%d/",time.localtime()), config.exp.name, "testPredictions/")
create_dirs([config.callbacks.testPredictions_dir, config.callbacks.tensorboard_log_dir, config.callbacks.checkpoint_dir])

print('Create the data generator.')
data_loader = ConvCifarDataLoader()
print('Create the model.')
model = MobileNetV2CifarModel(config)
print('Create the trainer')
trainer = SimpleCifarModelTrainer(model.model, data_loader.get_train_data(), config)
print('Start training the model.')
t0 = time.time()
trainer.train()
training_time=time.time()-t0

print("*******************************************************************************")
print(model.model.summary())


print("*******************************************************************************")
print("Training_time: ", training_time)


print("*******************************************************************************")
flops = get_flops(model.model, batch_size=1)
print(f"FLOPS: {flops / 10 ** 9:.03} G")


print("*******************************************************************************")
print("Create the tester")
tester = simpleCifarTester(model.model, data_loader.get_test_data(), config, training_time, flops)     
print("Start testing the model.")
tester.test()

In [ ]:
from keras.utils.vis_utils import plot_model
plot_model(model.model, to_file=config.exp.name+'.png', show_shapes=True, show_layer_names=True)

In [ ]:
print("*******************************************************************************")
# Plot the loss and accuracy curves for training and validation 
fig, ax = plt.subplots(2,1)
ax[0].plot(trainer.loss, color='b', label="Training loss")
ax[0].plot(trainer.val_loss, color='r', label="validation loss",axes =ax[0])
legend = ax[0].legend(loc='best', shadow=True)
ax[1].plot(trainer.acc, color='b', label="Training accuracy")
ax[1].plot(trainer.val_acc, color='r',label="Validation accuracy")
legend = ax[1].legend(loc='best', shadow=True)
plt.savefig(config.exp.name+".png")

In [ ]:
print("*******************************************************************************")
print(tester.result)

print("*******************************************************************************")
plot_confusion_matrix(tester.result, classes=range(10), name=config.exp.name)

Inception

In [ ]:
class InceptionCifarModel():
    def __init__(self, config):
        self.config=config
        self.build_model()

    def build_model(self):
        self.model = Sequential()
        self.model.add(InceptionV3(include_top=False, weights=None, input_tensor=None, input_shape=(150, 150, 3), pooling=None))
        self.model.add(GlobalAveragePooling2D())
        self.model.add(Dense(128))
        self.model.add(Activation("relu"))
        self.model.add(BatchNormalization(axis=-1, momentum=0.99, epsilon=0.001, center=True, scale=True, beta_initializer='zeros', gamma_initializer='ones', moving_mean_initializer='zeros', moving_variance_initializer='ones', beta_regularizer=None, gamma_regularizer=None, beta_constraint=None, gamma_constraint=None))
        self.model.add(Dense(64))
        self.model.add(Activation("relu"))
        self.model.add(Dense(32))
        self.model.add(Activation("relu"))
        self.model.add(Dense(10))
        self.model.add(Activation("softmax"))

        self.model.compile(
              loss='sparse_categorical_crossentropy',
              optimizer=self.config.model.optimizer,
              metrics=['accuracy', 'top_k_categorical_accuracy'])
              #metrics=['accuracy'])

config.exp.name="inception_cifar_from_config"
config.callbacks.tensorboard_log_dir = os.path.join("experiments", time.strftime("%Y-%m-%d/",time.localtime()), config.exp.name, "logs/")
config.callbacks.checkpoint_dir = os.path.join("experiments", time.strftime("%Y-%m-%d/",time.localtime()), config.exp.name, "checkpoints/")
config.callbacks.testPredictions_dir=os.path.join("experiments", time.strftime("%Y-%m-%d/",time.localtime()), config.exp.name, "testPredictions/")
create_dirs([config.callbacks.testPredictions_dir, config.callbacks.tensorboard_log_dir, config.callbacks.checkpoint_dir])

print('Create the data generator.')
data_loader = ConvCifarDataLoader()
print('Create the model.')
model = InceptionCifarModel(config)
print('Create the trainer')
trainer = SimpleCifarModelTrainer(model.model, data_loader.get_train_data(), config)
print('Start training the model.')
t0 = time.time()
trainer.train()
training_time=time.time()-t0

print("*******************************************************************************")
print(model.model.summary())


print("*******************************************************************************")
print("Training_time: ", training_time)


print("*******************************************************************************")
flops = get_flops(model.model, batch_size=1)
print(f"FLOPS: {flops / 10 ** 9:.03} G")


print("*******************************************************************************")
print("Create the tester")
tester = simpleCifarTester(model.model, data_loader.get_test_data(), config, training_time, flops)     
print("Start testing the model.")
tester.test()

In [ ]:
from keras.utils.vis_utils import plot_model
plot_model(model.model, to_file=config.exp.name+'.png', show_shapes=True, show_layer_names=True)

In [ ]:
print("*******************************************************************************")
# Plot the loss and accuracy curves for training and validation 
fig, ax = plt.subplots(2,1)
ax[0].plot(trainer.loss, color='b', label="Training loss")
ax[0].plot(trainer.val_loss, color='r', label="validation loss",axes =ax[0])
legend = ax[0].legend(loc='best', shadow=True)
ax[1].plot(trainer.acc, color='b', label="Training accuracy")
ax[1].plot(trainer.val_acc, color='r',label="Validation accuracy")
legend = ax[1].legend(loc='best', shadow=True)
plt.savefig(config.exp.name+".png")

In [ ]:
print("*******************************************************************************")
print(tester.result)

print("*******************************************************************************")
plot_confusion_matrix(tester.result, classes=range(10), name=config.exp.name)

DenseNet 121

In [ ]:
from tensorflow.keras.applications import DenseNet121
class DenseNet121CifarModel():
    def __init__(self, config):
        self.config=config
        self.build_model()

    def build_model(self):
        self.model = Sequential()
        self.model.add(DenseNet121(include_top=False, weights=None, input_tensor=None, input_shape=(150, 150, 3), pooling=None))
        self.model.add(GlobalAveragePooling2D())
        self.model.add(Dense(128))
        self.model.add(Activation("relu"))
        self.model.add(BatchNormalization(axis=-1, momentum=0.99, epsilon=0.001, center=True, scale=True, beta_initializer='zeros', gamma_initializer='ones', moving_mean_initializer='zeros', moving_variance_initializer='ones', beta_regularizer=None, gamma_regularizer=None, beta_constraint=None, gamma_constraint=None))
        self.model.add(Dense(64))
        self.model.add(Activation("relu"))
        self.model.add(Dense(32))
        self.model.add(Activation("relu"))
        self.model.add(Dense(10))
        self.model.add(Activation("softmax"))

        self.model.compile(
              loss='sparse_categorical_crossentropy',
              optimizer=self.config.model.optimizer,
              metrics=['accuracy', 'top_k_categorical_accuracy'])
              #metrics=['accuracy'])

config.exp.name="denseNet121_cifar_from_config"
config.callbacks.tensorboard_log_dir = os.path.join("experiments", time.strftime("%Y-%m-%d/",time.localtime()), config.exp.name, "logs/")
config.callbacks.checkpoint_dir = os.path.join("experiments", time.strftime("%Y-%m-%d/",time.localtime()), config.exp.name, "checkpoints/")
config.callbacks.testPredictions_dir=os.path.join("experiments", time.strftime("%Y-%m-%d/",time.localtime()), config.exp.name, "testPredictions/")
create_dirs([config.callbacks.testPredictions_dir, config.callbacks.tensorboard_log_dir, config.callbacks.checkpoint_dir])

print('Create the data generator.')
data_loader = ConvCifarDataLoader()
print('Create the model.')
model = DenseNet121CifarModel(config)
print('Create the trainer')
trainer = SimpleCifarModelTrainer(model.model, data_loader.get_train_data(), config)
print('Start training the model.')
t0 = time.time()
trainer.train()
training_time=time.time()-t0

print("*******************************************************************************")
print(model.model.summary())


print("*******************************************************************************")
print("Training_time: ", training_time)


print("*******************************************************************************")
flops = get_flops(model.model, batch_size=1)
print(f"FLOPS: {flops / 10 ** 9:.03} G")


print("*******************************************************************************")
print("Create the tester")
tester = simpleCifarTester(model.model, data_loader.get_test_data(), config, training_time, flops)     
print("Start testing the model.")
tester.test()

In [ ]:
from keras.utils.vis_utils import plot_model
plot_model(model.model, to_file=config.exp.name+'.png', show_shapes=True, show_layer_names=True)

In [ ]:
print("*******************************************************************************")
# Plot the loss and accuracy curves for training and validation 
fig, ax = plt.subplots(2,1)
ax[0].plot(trainer.loss, color='b', label="Training loss")
ax[0].plot(trainer.val_loss, color='r', label="validation loss",axes =ax[0])
legend = ax[0].legend(loc='best', shadow=True)
ax[1].plot(trainer.acc, color='b', label="Training accuracy")
ax[1].plot(trainer.val_acc, color='r',label="Validation accuracy")
legend = ax[1].legend(loc='best', shadow=True)
plt.savefig(config.exp.name+".png")

In [ ]:
print("*******************************************************************************")
print(tester.result)

print("*******************************************************************************")
plot_confusion_matrix(tester.result, classes=range(10), name=config.exp.name)

DenseNet 169

In [ ]:
from tensorflow.keras.applications import DenseNet169
class DenseNet169CifarModel():
    def __init__(self, config):
        self.config=config
        self.build_model()

    def build_model(self):
        self.model = Sequential()
        self.model.add(DenseNet169(include_top=False, weights=None, input_tensor=None, input_shape=(150, 150, 3), pooling=None))
        self.model.add(GlobalAveragePooling2D())
        self.model.add(Dense(128))
        self.model.add(Activation("relu"))
        self.model.add(BatchNormalization(axis=-1, momentum=0.99, epsilon=0.001, center=True, scale=True, beta_initializer='zeros', gamma_initializer='ones', moving_mean_initializer='zeros', moving_variance_initializer='ones', beta_regularizer=None, gamma_regularizer=None, beta_constraint=None, gamma_constraint=None))
        self.model.add(Dense(64))
        self.model.add(Activation("relu"))
        self.model.add(Dense(32))
        self.model.add(Activation("relu"))
        self.model.add(Dense(10))
        self.model.add(Activation("softmax"))

        self.model.compile(
              loss='sparse_categorical_crossentropy',
              optimizer=self.config.model.optimizer,
              metrics=['accuracy', 'top_k_categorical_accuracy'])
              #metrics=['accuracy'])

config.exp.name="denseNet169_cifar_from_config"
config.callbacks.tensorboard_log_dir = os.path.join("experiments", time.strftime("%Y-%m-%d/",time.localtime()), config.exp.name, "logs/")
config.callbacks.checkpoint_dir = os.path.join("experiments", time.strftime("%Y-%m-%d/",time.localtime()), config.exp.name, "checkpoints/")
config.callbacks.testPredictions_dir=os.path.join("experiments", time.strftime("%Y-%m-%d/",time.localtime()), config.exp.name, "testPredictions/")
create_dirs([config.callbacks.testPredictions_dir, config.callbacks.tensorboard_log_dir, config.callbacks.checkpoint_dir])

print('Create the data generator.')
data_loader = ConvCifarDataLoader()
print('Create the model.')
model = DenseNet169CifarModel(config)
print('Create the trainer')
trainer = SimpleCifarModelTrainer(model.model, data_loader.get_train_data(), config)
print('Start training the model.')
t0 = time.time()
trainer.train()
training_time=time.time()-t0

print("*******************************************************************************")
print(model.model.summary())


print("*******************************************************************************")
print("Training_time: ", training_time)


print("*******************************************************************************")
flops = get_flops(model.model, batch_size=1)
print(f"FLOPS: {flops / 10 ** 9:.03} G")


print("*******************************************************************************")
print("Create the tester")
tester = simpleCifarTester(model.model, data_loader.get_test_data(), config, training_time, flops)     
print("Start testing the model.")
tester.test()

In [ ]:
from keras.utils.vis_utils import plot_model
plot_model(model.model, to_file=config.exp.name+'.png', show_shapes=True, show_layer_names=True)

In [ ]:
print("*******************************************************************************")
# Plot the loss and accuracy curves for training and validation 
fig, ax = plt.subplots(2,1)
ax[0].plot(trainer.loss, color='b', label="Training loss")
ax[0].plot(trainer.val_loss, color='r', label="validation loss",axes =ax[0])
legend = ax[0].legend(loc='best', shadow=True)
ax[1].plot(trainer.acc, color='b', label="Training accuracy")
ax[1].plot(trainer.val_acc, color='r',label="Validation accuracy")
legend = ax[1].legend(loc='best', shadow=True)
plt.savefig(config.exp.name+".png")

In [ ]:
print("*******************************************************************************")
print(tester.result)

print("*******************************************************************************")
plot_confusion_matrix(tester.result, classes=range(10), name=config.exp.name)

NasNetMobile

In [ ]:
from tensorflow.keras.applications import NASNetMobile
class NASNetMobileCifarModel():
    def __init__(self, config):
        self.config=config
        self.build_model()

    def build_model(self):
        self.model = Sequential()
        self.model.add(NASNetMobile(include_top=False, weights=None, input_tensor=None, input_shape=(150, 150, 3), pooling=None))
        self.model.add(GlobalAveragePooling2D())
        self.model.add(Dense(128))
        self.model.add(Activation("relu"))
        self.model.add(BatchNormalization(axis=-1, momentum=0.99, epsilon=0.001, center=True, scale=True, beta_initializer='zeros', gamma_initializer='ones', moving_mean_initializer='zeros', moving_variance_initializer='ones', beta_regularizer=None, gamma_regularizer=None, beta_constraint=None, gamma_constraint=None))
        self.model.add(Dense(64))
        self.model.add(Activation("relu"))
        self.model.add(Dense(32))
        self.model.add(Activation("relu"))
        self.model.add(Dense(10))
        self.model.add(Activation("softmax"))

        self.model.compile(
              loss='sparse_categorical_crossentropy',
              optimizer=self.config.model.optimizer,
              metrics=['accuracy', 'top_k_categorical_accuracy'])
              #metrics=['accuracy'])

config.exp.name="nasNetMobile_cifar_from_config"
config.callbacks.tensorboard_log_dir = os.path.join("experiments", time.strftime("%Y-%m-%d/",time.localtime()), config.exp.name, "logs/")
config.callbacks.checkpoint_dir = os.path.join("experiments", time.strftime("%Y-%m-%d/",time.localtime()), config.exp.name, "checkpoints/")
config.callbacks.testPredictions_dir=os.path.join("experiments", time.strftime("%Y-%m-%d/",time.localtime()), config.exp.name, "testPredictions/")
create_dirs([config.callbacks.testPredictions_dir, config.callbacks.tensorboard_log_dir, config.callbacks.checkpoint_dir])

print('Create the data generator.')
data_loader = ConvCifarDataLoader()
print('Create the model.')
model = NASNetMobileCifarModel(config)
print('Create the trainer')
trainer = SimpleCifarModelTrainer(model.model, data_loader.get_train_data(), config)
print('Start training the model.')
t0 = time.time()
trainer.train()
training_time=time.time()-t0

print("*******************************************************************************")
print(model.model.summary())


print("*******************************************************************************")
print("Training_time: ", training_time)


print("*******************************************************************************")
flops = get_flops(model.model, batch_size=1)
print(f"FLOPS: {flops / 10 ** 9:.03} G")


print("*******************************************************************************")
print("Create the tester")
tester = simpleCifarTester(model.model, data_loader.get_test_data(), config, training_time, flops)     
print("Start testing the model.")
tester.test()

In [ ]:
from keras.utils.vis_utils import plot_model
plot_model(model.model, to_file=config.exp.name+'.png', show_shapes=True, show_layer_names=True)

In [ ]:
print("*******************************************************************************")
# Plot the loss and accuracy curves for training and validation 
fig, ax = plt.subplots(2,1)
ax[0].plot(trainer.loss, color='b', label="Training loss")
ax[0].plot(trainer.val_loss, color='r', label="validation loss",axes =ax[0])
legend = ax[0].legend(loc='best', shadow=True)
ax[1].plot(trainer.acc, color='b', label="Training accuracy")
ax[1].plot(trainer.val_acc, color='r',label="Validation accuracy")
legend = ax[1].legend(loc='best', shadow=True)
plt.savefig(config.exp.name+".png")

In [ ]:
print("*******************************************************************************")
print(tester.result)

print("*******************************************************************************")
plot_confusion_matrix(tester.result, classes=range(10), name=config.exp.name)

EfficientNet B0

In [ ]:
from tensorflow.keras.applications import EfficientNetB0
class EfficientNetB0CifarModel():
    def __init__(self, config):
        self.config=config
        self.build_model()

    def build_model(self):
        self.model = Sequential()
        self.model.add(EfficientNetB0(include_top=False, weights=None, input_tensor=None, input_shape=(150, 150, 3), pooling=None))
        self.model.add(GlobalAveragePooling2D())
        self.model.add(Dense(128))
        self.model.add(Activation("relu"))
        self.model.add(BatchNormalization(axis=-1, momentum=0.99, epsilon=0.001, center=True, scale=True, beta_initializer='zeros', gamma_initializer='ones', moving_mean_initializer='zeros', moving_variance_initializer='ones', beta_regularizer=None, gamma_regularizer=None, beta_constraint=None, gamma_constraint=None))
        self.model.add(Dense(64))
        self.model.add(Activation("relu"))
        self.model.add(Dense(32))
        self.model.add(Activation("relu"))
        self.model.add(Dense(10))
        self.model.add(Activation("softmax"))

        self.model.compile(
              loss='sparse_categorical_crossentropy',
              optimizer=self.config.model.optimizer,
              metrics=['accuracy', 'top_k_categorical_accuracy'])
              #metrics=['accuracy'])

config.exp.name="efficientNetB0_cifar_from_config"
config.callbacks.tensorboard_log_dir = os.path.join("experiments", time.strftime("%Y-%m-%d/",time.localtime()), config.exp.name, "logs/")
config.callbacks.checkpoint_dir = os.path.join("experiments", time.strftime("%Y-%m-%d/",time.localtime()), config.exp.name, "checkpoints/")
config.callbacks.testPredictions_dir=os.path.join("experiments", time.strftime("%Y-%m-%d/",time.localtime()), config.exp.name, "testPredictions/")
create_dirs([config.callbacks.testPredictions_dir, config.callbacks.tensorboard_log_dir, config.callbacks.checkpoint_dir])

print('Create the data generator.')
data_loader = ConvCifarDataLoader()
print('Create the model.')
model = EfficientNetB0CifarModel(config)
print('Create the trainer')
trainer = SimpleCifarModelTrainer(model.model, data_loader.get_train_data(), config)
print('Start training the model.')
t0 = time.time()
trainer.train()
training_time=time.time()-t0

print("*******************************************************************************")
print(model.model.summary())


print("*******************************************************************************")
print("Training_time: ", training_time)


print("*******************************************************************************")
flops = get_flops(model.model, batch_size=1)
print(f"FLOPS: {flops / 10 ** 9:.03} G")


print("*******************************************************************************")
print("Create the tester")
tester = simpleCifarTester(model.model, data_loader.get_test_data(), config, training_time, flops)     
print("Start testing the model.")
tester.test()

In [ ]:
from keras.utils.vis_utils import plot_model
plot_model(model.model, to_file=config.exp.name+'.png', show_shapes=True, show_layer_names=True)

In [ ]:
print("*******************************************************************************")
# Plot the loss and accuracy curves for training and validation 
fig, ax = plt.subplots(2,1)
ax[0].plot(trainer.loss, color='b', label="Training loss")
ax[0].plot(trainer.val_loss, color='r', label="validation loss",axes =ax[0])
legend = ax[0].legend(loc='best', shadow=True)
ax[1].plot(trainer.acc, color='b', label="Training accuracy")
ax[1].plot(trainer.val_acc, color='r',label="Validation accuracy")
legend = ax[1].legend(loc='best', shadow=True)
plt.savefig(config.exp.name+".png")

In [ ]:
print("*******************************************************************************")
print(tester.result)

print("*******************************************************************************")
plot_confusion_matrix(tester.result, classes=range(10), name=config.exp.name)

EfficientNetB1

In [ ]:
from tensorflow.keras.applications import EfficientNetB1
class EfficientNetB1CifarModel():
    def __init__(self, config):
        self.config=config
        self.build_model()

    def build_model(self):
        self.model = Sequential()
        self.model.add(EfficientNetB1(include_top=False, weights=None, input_tensor=None, input_shape=(150, 150, 3), pooling=None))
        self.model.add(GlobalAveragePooling2D())
        self.model.add(Dense(128))
        self.model.add(Activation("relu"))
        self.model.add(BatchNormalization(axis=-1, momentum=0.99, epsilon=0.001, center=True, scale=True, beta_initializer='zeros', gamma_initializer='ones', moving_mean_initializer='zeros', moving_variance_initializer='ones', beta_regularizer=None, gamma_regularizer=None, beta_constraint=None, gamma_constraint=None))
        self.model.add(Dense(64))
        self.model.add(Activation("relu"))
        self.model.add(Dense(32))
        self.model.add(Activation("relu"))
        self.model.add(Dense(10))
        self.model.add(Activation("softmax"))

        self.model.compile(
              loss='sparse_categorical_crossentropy',
              optimizer=self.config.model.optimizer,
              metrics=['accuracy', 'top_k_categorical_accuracy'])
              #metrics=['accuracy'])

config.exp.name="efficientNetB1_cifar_from_config"
config.callbacks.tensorboard_log_dir = os.path.join("experiments", time.strftime("%Y-%m-%d/",time.localtime()), config.exp.name, "logs/")
config.callbacks.checkpoint_dir = os.path.join("experiments", time.strftime("%Y-%m-%d/",time.localtime()), config.exp.name, "checkpoints/")
config.callbacks.testPredictions_dir=os.path.join("experiments", time.strftime("%Y-%m-%d/",time.localtime()), config.exp.name, "testPredictions/")
create_dirs([config.callbacks.testPredictions_dir, config.callbacks.tensorboard_log_dir, config.callbacks.checkpoint_dir])

print('Create the data generator.')
data_loader = ConvCifarDataLoader()
print('Create the model.')
model = EfficientNetB1CifarModel(config)
print('Create the trainer')
trainer = SimpleCifarModelTrainer(model.model, data_loader.get_train_data(), config)
print('Start training the model.')
t0 = time.time()
trainer.train()
training_time=time.time()-t0

print("*******************************************************************************")
print(model.model.summary())


print("*******************************************************************************")
print("Training_time: ", training_time)


print("*******************************************************************************")
flops = get_flops(model.model, batch_size=1)
print(f"FLOPS: {flops / 10 ** 9:.03} G")


print("*******************************************************************************")
print("Create the tester")
tester = simpleCifarTester(model.model, data_loader.get_test_data(), config, training_time, flops)     
print("Start testing the model.")
tester.test()

In [ ]:
from keras.utils.vis_utils import plot_model
plot_model(model.model, to_file=config.exp.name+'.png', show_shapes=True, show_layer_names=True)

In [ ]:
print("*******************************************************************************")
# Plot the loss and accuracy curves for training and validation 
fig, ax = plt.subplots(2,1)
ax[0].plot(trainer.loss, color='b', label="Training loss")
ax[0].plot(trainer.val_loss, color='r', label="validation loss",axes =ax[0])
legend = ax[0].legend(loc='best', shadow=True)
ax[1].plot(trainer.acc, color='b', label="Training accuracy")
ax[1].plot(trainer.val_acc, color='r',label="Validation accuracy")
legend = ax[1].legend(loc='best', shadow=True)
plt.savefig(config.exp.name+".png")

In [ ]:
print("*******************************************************************************")
print(tester.result)

print("*******************************************************************************")
plot_confusion_matrix(tester.result, classes=range(10), name=config.exp.name)

EfficientNet B2

In [ ]:
from tensorflow.keras.applications import EfficientNetB2
class EfficientNetB2CifarModel():
    def __init__(self, config):
        self.config=config
        self.build_model()

    def build_model(self):
        self.model = Sequential()
        self.model.add(EfficientNetB2(include_top=False, weights=None, input_tensor=None, input_shape=(150, 150, 3), pooling=None))
        self.model.add(GlobalAveragePooling2D())
        self.model.add(Dense(128))
        self.model.add(Activation("relu"))
        self.model.add(BatchNormalization(axis=-1, momentum=0.99, epsilon=0.001, center=True, scale=True, beta_initializer='zeros', gamma_initializer='ones', moving_mean_initializer='zeros', moving_variance_initializer='ones', beta_regularizer=None, gamma_regularizer=None, beta_constraint=None, gamma_constraint=None))
        self.model.add(Dense(64))
        self.model.add(Activation("relu"))
        self.model.add(Dense(32))
        self.model.add(Activation("relu"))
        self.model.add(Dense(10))
        self.model.add(Activation("softmax"))

        self.model.compile(
              loss='sparse_categorical_crossentropy',
              optimizer=self.config.model.optimizer,
              metrics=['accuracy', 'top_k_categorical_accuracy'])
              #metrics=['accuracy'])

config.exp.name="efficientNetB2_cifar_from_config"
config.callbacks.tensorboard_log_dir = os.path.join("experiments", time.strftime("%Y-%m-%d/",time.localtime()), config.exp.name, "logs/")
config.callbacks.checkpoint_dir = os.path.join("experiments", time.strftime("%Y-%m-%d/",time.localtime()), config.exp.name, "checkpoints/")
config.callbacks.testPredictions_dir=os.path.join("experiments", time.strftime("%Y-%m-%d/",time.localtime()), config.exp.name, "testPredictions/")
create_dirs([config.callbacks.testPredictions_dir, config.callbacks.tensorboard_log_dir, config.callbacks.checkpoint_dir])

print('Create the data generator.')
data_loader = ConvCifarDataLoader()
print('Create the model.')
model = EfficientNetB2CifarModel(config)
print('Create the trainer')
trainer = SimpleCifarModelTrainer(model.model, data_loader.get_train_data(), config)
print('Start training the model.')
t0 = time.time()
trainer.train()
training_time=time.time()-t0

print("*******************************************************************************")
print(model.model.summary())


print("*******************************************************************************")
print("Training_time: ", training_time)


print("*******************************************************************************")
flops = get_flops(model.model, batch_size=1)
print(f"FLOPS: {flops / 10 ** 9:.03} G")


print("*******************************************************************************")
print("Create the tester")
tester = simpleCifarTester(model.model, data_loader.get_test_data(), config, training_time, flops)     
print("Start testing the model.")
tester.test()

In [ ]:
from keras.utils.vis_utils import plot_model
plot_model(model.model, to_file=config.exp.name+'.png', show_shapes=True, show_layer_names=True)

In [ ]:
print("*******************************************************************************")
# Plot the loss and accuracy curves for training and validation 
fig, ax = plt.subplots(2,1)
ax[0].plot(trainer.loss, color='b', label="Training loss")
ax[0].plot(trainer.val_loss, color='r', label="validation loss",axes =ax[0])
legend = ax[0].legend(loc='best', shadow=True)
ax[1].plot(trainer.acc, color='b', label="Training accuracy")
ax[1].plot(trainer.val_acc, color='r',label="Validation accuracy")
legend = ax[1].legend(loc='best', shadow=True)
plt.savefig(config.exp.name+".png")

In [ ]:
print("*******************************************************************************")
print(tester.result)

print("*******************************************************************************")
plot_confusion_matrix(tester.result, classes=range(10), name=config.exp.name)

ShuffleNet

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Add,AveragePooling2D,Dense, AvgPool2D,BatchNormalization, ReLU, DepthwiseConv2D, Reshape, Permute,Conv2D, MaxPool2D, GlobalAveragePooling2D, concatenate
def channel_shuffle(x, groups):  
    _, width, height, channels = x.get_shape().as_list()
    group_ch = channels // groups

    x = Reshape([width, height, group_ch, groups])(x)
    x = Permute([1, 2, 4, 3])(x)
    x = Reshape([width, height, channels])(x)
    return x
def shuffle_unit(x, groups, channels,strides):

    y = x
    x = Conv2D(channels//4, kernel_size = 1, strides = (1,1),padding = 'same', groups=groups)(x)
    x = BatchNormalization()(x)
    x = ReLU()(x)

    x = channel_shuffle(x, groups)
    
    x = DepthwiseConv2D(kernel_size = (3,3), strides = strides, padding = 'same')(x)
    x = BatchNormalization()(x)

    if strides == (2,2):
        channels = channels - y.shape[-1]
    x = Conv2D(channels, kernel_size = 1, strides = (1,1),padding = 'same', groups=groups)(x)
    x = BatchNormalization()(x)

    if strides ==(1,1):
        x =Add()([x,y])
    if strides == (2,2):
        y = AvgPool2D((3,3), strides = (2,2), padding = 'same')(y)
        x = concatenate([x,y])
    
    x = ReLU()(x)


    return x
 
def Shuffle_Net(n_classes, start_channels ,input_shape = (224,224,3)):

    groups = 2
    input = Input (input_shape)

    x =  Conv2D (24,kernel_size=3,strides = (2,2), padding = 'same', use_bias = True)(input)
    x =  BatchNormalization()(x)
    x =  ReLU()(x)
    
    x = MaxPool2D (pool_size=(3,3), strides = 2, padding='same')(x)

    repetitions = [3,7,3]

    for i,repetition in enumerate(repetitions):
        channels = start_channels * (2**i)

        x  = shuffle_unit(x, groups, channels,strides = (2,2))

        for i in range(repetition):
            x = shuffle_unit(x, groups, channels,strides=(1,1))

    x = GlobalAveragePooling2D()(x)

    output = Dense(n_classes,activation='softmax')(x)

    model = Model(input, output)
    return model

In [ ]:
class ShuffleNetCifarModel():
    def __init__(self, config):
        self.config=config
        self.build_model()

    def build_model(self):
        self.model = Sequential()
        self.model.add(Shuffle_Net(n_classes=1000, start_channels=200, input_shape=(150, 150, 3)))
        #self.model.add(GlobalAveragePooling2D())
        self.model.add(Dense(128))
        self.model.add(Activation("relu"))
        self.model.add(BatchNormalization(axis=-1, momentum=0.99, epsilon=0.001, center=True, scale=True, beta_initializer='zeros', gamma_initializer='ones', moving_mean_initializer='zeros', moving_variance_initializer='ones', beta_regularizer=None, gamma_regularizer=None, beta_constraint=None, gamma_constraint=None))
        self.model.add(Dense(64))
        self.model.add(Activation("relu"))
        self.model.add(Dense(32))
        self.model.add(Activation("relu"))
        self.model.add(Dense(10))
        self.model.add(Activation("softmax"))

        self.model.compile(
              loss='sparse_categorical_crossentropy',
              optimizer=self.config.model.optimizer,
              metrics=['accuracy', 'top_k_categorical_accuracy'])
              #metrics=['accuracy'])

config.exp.name="shuffleNet_cifar_from_config"
config.callbacks.tensorboard_log_dir = os.path.join("experiments", time.strftime("%Y-%m-%d/",time.localtime()), config.exp.name, "logs/")
config.callbacks.checkpoint_dir = os.path.join("experiments", time.strftime("%Y-%m-%d/",time.localtime()), config.exp.name, "checkpoints/")
config.callbacks.testPredictions_dir=os.path.join("experiments", time.strftime("%Y-%m-%d/",time.localtime()), config.exp.name, "testPredictions/")
create_dirs([config.callbacks.testPredictions_dir, config.callbacks.tensorboard_log_dir, config.callbacks.checkpoint_dir])

print('Create the data generator.')
data_loader = ConvCifarDataLoader()
print('Create the model.')
model = ShuffleNetCifarModel(config)
print('Create the trainer')
trainer = SimpleCifarModelTrainer(model.model, data_loader.get_train_data(), config)
print('Start training the model.')
t0 = time.time()
trainer.train()
training_time=time.time()-t0

print("*******************************************************************************")
print(model.model.summary())


print("*******************************************************************************")
print("Training_time: ", training_time)


print("*******************************************************************************")
flops = get_flops(model.model, batch_size=1)
print(f"FLOPS: {flops / 10 ** 9:.03} G")


print("*******************************************************************************")
print("Create the tester")
tester = simpleCifarTester(model.model, data_loader.get_test_data(), config, training_time, flops)     
print("Start testing the model.")
tester.test()

In [ ]:
from keras.utils.vis_utils import plot_model
plot_model(model.model, to_file=config.exp.name+'.png', show_shapes=True, show_layer_names=True)

In [ ]:
print("*******************************************************************************")
# Plot the loss and accuracy curves for training and validation 
fig, ax = plt.subplots(2,1)
ax[0].plot(trainer.loss, color='b', label="Training loss")
ax[0].plot(trainer.val_loss, color='r', label="validation loss",axes =ax[0])
legend = ax[0].legend(loc='best', shadow=True)
ax[1].plot(trainer.acc, color='b', label="Training accuracy")
ax[1].plot(trainer.val_acc, color='r',label="Validation accuracy")
legend = ax[1].legend(loc='best', shadow=True)
plt.savefig(config.exp.name+".png")

In [ ]:
print("*******************************************************************************")
print(tester.result)

print("*******************************************************************************")
plot_confusion_matrix(tester.result, classes=range(10), name=config.exp.name)

SqueezeNet

In [ ]:
def SqueezeNet():  
  IMAGE_SIZE = [150, 150]
  bnmomemtum=0.9
  def fire(x, squeeze, expand):
    y  = tf.keras.layers.Conv2D(filters=squeeze, kernel_size=1, activation='relu', padding='same')(x)
    y = tf.keras.layers.BatchNormalization(momentum=bnmomemtum)(y)
    y1 = tf.keras.layers.Conv2D(filters=expand//2, kernel_size=1, activation='relu', padding='same')(y)
    y1 = tf.keras.layers.BatchNormalization(momentum=bnmomemtum)(y1)
    y3 = tf.keras.layers.Conv2D(filters=expand//2, kernel_size=3, activation='relu', padding='same')(y)
    y3 = tf.keras.layers.BatchNormalization(momentum=bnmomemtum)(y3)
    return tf.keras.layers.concatenate([y1, y3])

  def fire_module(squeeze, expand):
    return lambda x: fire(x, squeeze, expand)

  x = tf.keras.layers.Input(shape=[*IMAGE_SIZE, 3]) # input is 192x192 pixels RGB

  y = tf.keras.layers.Conv2D(kernel_size=3, filters=32, padding='same', use_bias=True, activation='relu')(x)
  y = tf.keras.layers.BatchNormalization(momentum=bnmomemtum)(y)
  y = fire_module(24, 48)(y)
  y = tf.keras.layers.MaxPooling2D(pool_size=2)(y)
  y = fire_module(48, 96)(y)
  y = tf.keras.layers.MaxPooling2D(pool_size=2)(y)
  y = fire_module(64, 128)(y)
  y = tf.keras.layers.MaxPooling2D(pool_size=2)(y)
  y = fire_module(48, 96)(y)
  y = tf.keras.layers.MaxPooling2D(pool_size=2)(y)
  y = fire_module(24, 48)(y)
  y = tf.keras.layers.GlobalAveragePooling2D()(y)
  y = tf.keras.layers.Dense(1000, activation='softmax')(y)

  model = tf.keras.Model(x, y)

  return(model)

In [ ]:
class SqueezeNetCifarModel():
    def __init__(self, config):
        self.config=config
        self.build_model()

    def build_model(self):
        self.model = Sequential()
        self.model.add(SqueezeNet())
        self.model.add(Dense(128))
        self.model.add(Activation("relu"))
        self.model.add(BatchNormalization(axis=-1, momentum=0.99, epsilon=0.001, center=True, scale=True, beta_initializer='zeros', gamma_initializer='ones', moving_mean_initializer='zeros', moving_variance_initializer='ones', beta_regularizer=None, gamma_regularizer=None, beta_constraint=None, gamma_constraint=None))
        self.model.add(Dense(64))
        self.model.add(Activation("relu"))
        self.model.add(Dense(32))
        self.model.add(Activation("relu"))
        self.model.add(Dense(10))
        self.model.add(Activation("softmax"))

        self.model.compile(
              loss='sparse_categorical_crossentropy',
              optimizer=self.config.model.optimizer,
              metrics=['accuracy', 'top_k_categorical_accuracy'])
              #metrics=['accuracy'])

config.exp.name="squeezeNet_cifar_from_config"
config.callbacks.tensorboard_log_dir = os.path.join("experiments", time.strftime("%Y-%m-%d/",time.localtime()), config.exp.name, "logs/")
config.callbacks.checkpoint_dir = os.path.join("experiments", time.strftime("%Y-%m-%d/",time.localtime()), config.exp.name, "checkpoints/")
config.callbacks.testPredictions_dir=os.path.join("experiments", time.strftime("%Y-%m-%d/",time.localtime()), config.exp.name, "testPredictions/")
create_dirs([config.callbacks.testPredictions_dir, config.callbacks.tensorboard_log_dir, config.callbacks.checkpoint_dir])

print('Create the data generator.')
data_loader = ConvCifarDataLoader()
print('Create the model.')
model = SqueezeNetCifarModel(config)
print('Create the trainer')
trainer = SimpleCifarModelTrainer(model.model, data_loader.get_train_data(), config)
print('Start training the model.')
t0 = time.time()
trainer.train()
training_time=time.time()-t0

print("*******************************************************************************")
print(model.model.summary())


print("*******************************************************************************")
print("Training_time: ", training_time)


print("*******************************************************************************")
flops = get_flops(model.model, batch_size=1)
print(f"FLOPS: {flops / 10 ** 9:.03} G")


print("*******************************************************************************")
print("Create the tester")
tester = simpleCifarTester(model.model, data_loader.get_test_data(), config, training_time, flops)     
print("Start testing the model.")
tester.test()

In [ ]:
from keras.utils.vis_utils import plot_model
plot_model(model.model, to_file=config.exp.name+'.png', show_shapes=True, show_layer_names=True)

In [ ]:
print("*******************************************************************************")
# Plot the loss and accuracy curves for training and validation 
fig, ax = plt.subplots(2,1)
ax[0].plot(trainer.loss, color='b', label="Training loss")
ax[0].plot(trainer.val_loss, color='r', label="validation loss",axes =ax[0])
legend = ax[0].legend(loc='best', shadow=True)
ax[1].plot(trainer.acc, color='b', label="Training accuracy")
ax[1].plot(trainer.val_acc, color='r',label="Validation accuracy")
legend = ax[1].legend(loc='best', shadow=True)
plt.savefig(config.exp.name+".png")

In [ ]:
print("*******************************************************************************")
print(tester.result)

print("*******************************************************************************")
plot_confusion_matrix(tester.result, classes=range(10), name=config.exp.name)